In [1]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, RANSACRegressor

from scipy import stats

import joblib

import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors

from netCDF4 import Dataset, num2date, date2num
import os

import seaborn as sns
import cmocean as cmocn

%matplotlib inline

In [2]:
#fname = '../generalStatistics/goodData_QSwithTAO_with2hrStatsConv.nc'
fname = '../generalStatistics/rainFlagRemovedBuoyDataBadQualityRemovedMatchupAmbuguitiesAdded.nc'
ds = xr.open_dataset(fname)
ds['mean_WVEL'] = np.sqrt(ds['mean_U10N_x'] **2 + ds['mean_U10N_y']**2)

In [3]:
data = ds.to_dataframe()
df = data.copy()

In [4]:
ds

<xarray.Dataset>
Dimensions:                                            (count: 266786)
Coordinates:
  * count                                              (count) int64 0 ... 29...
Data variables: (12/89)
    Wind Direction (TAO)                               (count) float32 140.6 ...
    WDIR_QC                                            (count) float32 2.0 .....
    WDIR_DM                                            (count) |S1 b'D' ... b'D'
    Wind Speed (TAO)                                   (count) float32 6.79 ....
    WSPD_QC                                            (count) float32 2.0 .....
    WSPD_DM                                            (count) |S1 b'D' ... b'D'
    ...                                                 ...
    Direction Difference (QuikSCAT - TAO)              (count) float64 0.1288...
    cos(Direction Difference (QuikSCAT - TAO))         (count) float64 1.0 .....
    sin(Direction Difference (QuikSCAT - TAO))         (count) float64 0.0022...
    Zonal Wind Speed Difference (QuikSCAT - TAO)       (count) float64 0.1173...
    Meridional Wind Speed Difference (QuikSCAT - TAO)  (count) float64 -0.117...
    mean_WVEL                                          (count) float64 6.766 ...

In [5]:
selectX = ['Speed Difference (QuikSCAT - TAO)',
          'cos(Direction Difference (QuikSCAT - TAO))',
          'sin(Direction Difference (QuikSCAT - TAO))']

X = df[selectX]

normX = (X - X.mean(axis=0))/ X.std(axis=0)

dbscan = DBSCAN(eps=0.15, min_samples= 500)
df['label'] = dbscan.fit_predict(normX)

In [6]:
# load trained model 
classifier = joblib.load("../../classification_machineLearning/randomForest/random_forest_rainDetection.joblib")

# Features used for classification
features = ['mean_WSPD', 'mean_SST', 'mean_AIRT', 'mean_SST - AIRT', 'mean_RELH', 'std_WSPD', 'std_cosWDIR', 'std_sinWDIR', 'std_SST', 'std_AIRT', 'std_SST - AIRT', 'std_RELH']
X = df[features]

#predict rain flags
y_pred = classifier.predict(X)
df['predicted_rain_flag'] = y_pred

In [7]:
def plotSusanLikeWithoutCorrectedAngle(SUBDF, lat, lon, suffix='all'):
    subDF = SUBDF.loc[SUBDF['LATITUDE'] == lat]
    subDF = subDF.loc[SUBDF['LONGITUDE'] == (lon+360)%360]

    if len(subDF.index) <1:
        return
    
    Ux_QS = np.array(subDF['Zonal Wind Speed (QuikSCAT)'])
    Uy_QS = np.array(subDF['Meridional Wind Speed (QuikSCAT)'])
    
    Ux_TAO = np.array(subDF['Zonal Neutral Wind Speed (TAO)'])
    Uy_TAO = np.array(subDF['Meridional Neutral Wind Speed (TAO)'])
    
    QS = Ux_QS + 1j * Uy_QS
    TAO = Ux_TAO + 1j *Uy_TAO
    
    r = QS/TAO
    angle = np.angle(r, deg=True)
    ratio = np.abs(r)
    subDF['angle_diff'] = angle
    subDF['ratio'] = ratio
    
    deployments = subDF['Deployment index']
    dep_labels = np.unique(np.array(deployments))
    startDates = []
    endDates = []
        
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(20,8))
    
    for dep in dep_labels:
        ssubDF = subDF.loc[subDF['Deployment index'] == dep]
        
        startDates.append(np.min(ssubDF['QS_TIME']))
        endDates.append(np.max(ssubDF['QS_TIME']))
        
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m'))
    
        y = np.array(ssubDF['ratio'])

        mask = np.logical_or(np.isnan(y), abs(y) > 100)
        x = x[~mask].reshape(-1,1)
        y = y[~mask].reshape(-1,1)

        if len(x) <2:
            continue
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        
        axes[0].plot(ssubDF['QS_TIME'][~mask], Y, color='k')
    
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m'))
    
        y = np.array(ssubDF['angle_diff'])

        mask = np.logical_or(np.isnan(y), abs(y) > 100)
        x = x[~mask].reshape(-1,1)
        y = y[~mask].reshape(-1,1)

        if len(x) <2:
            continue
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        
        axes[1].plot(ssubDF['QS_TIME'][~mask], Y, color='k')
        
    ax = axes[0]
    ax.scatter(subDF['QS_TIME'], ratio, c = 'r', s = 0.2)
    
    ax = axes[1]
    ax.scatter(subDF['QS_TIME'], angle, c = 'r', s = 0.2)
    
    for i in range(len(startDates)):
        colorR =list(np.random.choice(range(256), size=3)/256)
        
        axes[0].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
        axes[1].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
    
    axes[0].set_ylim(0,2)
    axes[1].set_ylim(-50,50)
    
    axes[0].set_title('Ratio of wind speeds (QS/TAO)')
    axes[1].set_title('Difference in wind direction (QS - TAO)')

    axes[0].grid(axis='y')
    axes[1].grid(axis='y')
    
    if lat < 0:
        latUnits = 'S'
        lat = abs(lat)
    else:
        latUnits = 'N'
    
    if lon < 0:
        lonUnits = 'W'
        lon = abs(lon) # 360
    else:
        lonUnits = 'E'
    title = f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_{suffix}.png'
    plt.savefig(title, dpi = 100)
    plt.close()

In [8]:
def plotSusanLikeWithCorrectedAngle(SUBDF, lat, lon, suffix='all'):
    subDF = SUBDF.loc[SUBDF['LATITUDE'] == lat]
    subDF = subDF.loc[SUBDF['LONGITUDE'] == (lon+360)%360]

    if len(subDF.index) <1:
        return
    
    Ux_QS = np.array(subDF['Zonal Wind Speed (QuikSCAT)'])
    Uy_QS = np.array(subDF['Meridional Wind Speed (QuikSCAT)'])
    
    Ux_TAO = np.array(subDF['Zonal Neutral Wind Speed (TAO)'])
    Uy_TAO = np.array(subDF['Meridional Neutral Wind Speed (TAO)'])
    
    QS = Ux_QS + 1j * Uy_QS
    TAO = Ux_TAO + 1j *Uy_TAO
    
    r = QS/TAO
    angle = np.angle(r, deg=True)
    ratio = np.abs(r)
    subDF['angle_diff'] = angle
    subDF['ratio'] = ratio

    subDF['corrected_angle_diff'] = angle
    subDF['Zonal Neutral Wind Speed (TAO) aft. correc.'] = subDF['Zonal Neutral Wind Speed (TAO)']
    subDF['Meridional Neutral Wind Speed (TAO) aft. correc.'] = subDF['Meridional Neutral Wind Speed (TAO)']

    mask = np.logical_or(np.isnan(ratio), abs(ratio) > 100)
    subDF = subDF.loc[~mask]
    
    deployments = subDF['Deployment index']
    dep_labels = np.unique(np.array(deployments))
    startDates = []
    endDates = []
        
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20,12))
    
    for dep in dep_labels:
        ssubDF = subDF.loc[subDF['Deployment index'] == dep]
        
        startDates.append(np.min(ssubDF['QS_TIME']))
        endDates.append(np.max(ssubDF['QS_TIME']))
        
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m')).reshape(-1,1)    
        y = np.array(ssubDF['ratio']).reshape(-1,1)

        if len(x) <2:
            continue
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        axes[0].plot(ssubDF['QS_TIME'], Y, color='k')    
        y = np.array(ssubDF['angle_diff']).reshape(-1,1)
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        axes[1].plot(ssubDF['QS_TIME'], Y, color='k')
        
        y -= (Y - np.mean(Y)) 
        ransac.fit(x, y)
        Y = ransac.predict(x)
        axes[2].plot(ssubDF['QS_TIME'], Y, color='k')
        
        # ssubDF.loc[:,'corrected_angle_diff'] = y[:,0]
        # ssubDF.loc[:,'corrected_angle'] = ssubDF['Wind Direction (QuikSCAT)'] - ssubDF['corrected_angle_diff']

        # Ux_TAO = ssubDF['Neutral Wind Speed at 10m (TAO)'] * np.cos(np.deg2rad(ssubDF['corrected_angle']))
        # Uy_TAO = ssubDF['Neutral Wind Speed at 10m (TAO)'] * np.sin(np.deg2rad(ssubDF['corrected_angle']))
        # subDF.loc[subDF['Deployment index'] == dep, 'Zonal Neutral Wind Speed (TAO) aft. correc.'] = Ux_TAO
        # subDF.loc[subDF['Deployment index'] == dep, 'Meridional Neutral Wind Speed (TAO) aft. correc.'] = Uy_TAO
        subDF.loc[subDF['Deployment index'] == dep, 'corrected_angle_diff'] = y[:,0]
        
    ax = axes[0]
    ax.scatter(subDF['QS_TIME'], subDF['ratio'], c = 'r', s = 0.2)
    
    ax = axes[1]
    ax.scatter(subDF['QS_TIME'], subDF['angle_diff'], c = 'r', s = 0.2)

    ax = axes[2]
    ax.scatter(subDF['QS_TIME'], subDF['corrected_angle_diff'], c = 'r', s = 0.2)

    
    for i in range(len(startDates)):
        colorR =list(np.random.choice(range(256), size=3)/256)
        
        axes[0].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
        axes[1].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
    
    axes[0].set_ylim(0,2)
    axes[1].set_ylim(-50,50)
    axes[2].set_ylim(-50,50)
    
    axes[0].set_title('Ratio of wind speeds (QS/TAO)')
    axes[1].set_title('Difference in wind direction (QS - TAO)')
    axes[2].set_title('Difference in wind direction (QS - TAO) after correction from robust regression fit')

    axes[0].grid(axis='y')
    axes[1].grid(axis='y')
    axes[2].grid(axis='y')
    
    if lat < 0:
        latUnits = 'S'
        lat = abs(lat)
    else:
        latUnits = 'N'
    
    if lon < 0:
        lonUnits = 'W'
        lon = abs(lon) # 360
    else:
        lonUnits = 'E'
    title = f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_{suffix}.png'
    plt.savefig(title, dpi = 100)
    plt.close()

In [9]:
def plotSusanLikeWithCorrectedAngleGreyBad(SUBDF, lat, lon, suffix='all'):
    subDF = SUBDF.loc[SUBDF['LATITUDE'] == lat]
    subDF = subDF.loc[SUBDF['LONGITUDE'] == (lon+360)%360]

    if len(subDF.index) <1:
        return
    
    Ux_QS = np.array(subDF['Zonal Wind Speed (QuikSCAT)'])
    Uy_QS = np.array(subDF['Meridional Wind Speed (QuikSCAT)'])
    
    Ux_TAO = np.array(subDF['Zonal Neutral Wind Speed (TAO)'])
    Uy_TAO = np.array(subDF['Meridional Neutral Wind Speed (TAO)'])
    
    QS = Ux_QS + 1j * Uy_QS
    TAO = Ux_TAO + 1j *Uy_TAO
    
    r = QS/TAO
    angle = np.angle(r, deg=True)
    ratio = np.abs(r)
    subDF['angle_diff'] = angle
    subDF['ratio'] = ratio

    subDF['corrected_angle_diff'] = angle
    subDF['Zonal Neutral Wind Speed (TAO) aft. correc.'] = subDF['Zonal Neutral Wind Speed (TAO)']
    subDF['Meridional Neutral Wind Speed (TAO) aft. correc.'] = subDF['Meridional Neutral Wind Speed (TAO)']

    mask = np.logical_or(np.isnan(ratio), abs(ratio) > 100)
    subDF = subDF.loc[~mask]


    subDF2 = subDF.loc[subDF['label'] == -1]
    subDF = subDF.loc[subDF['label'] == 0]
    
    deployments = subDF['Deployment index']
    dep_labels = np.unique(np.array(deployments))
    startDates = []
    endDates = []
        
    fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20,12))
    
    for dep in dep_labels:
        ssubDF = subDF.loc[subDF['Deployment index'] == dep]
        
        startDates.append(np.min(ssubDF['QS_TIME']))
        endDates.append(np.max(ssubDF['QS_TIME']))
        
        x = ((np.array(ssubDF['QS_TIME']) - np.datetime64('2000-01-01T00:00:00'))/np.timedelta64(1,'m')).reshape(-1,1)    
        y = np.array(ssubDF['ratio']).reshape(-1,1)

        if len(x) <2:
            continue
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        axes[0].plot(ssubDF['QS_TIME'], Y, color='k')    
        y = np.array(ssubDF['angle_diff']).reshape(-1,1)
    
        ransac = RANSACRegressor(random_state=42)
        ransac.fit(x, y)
        Y = ransac.predict(x)
        axes[1].plot(ssubDF['QS_TIME'], Y, color='k')
        
        y -= (Y - np.mean(Y)) 
        ransac.fit(x, y)
        Y = ransac.predict(x)
        axes[2].plot(ssubDF['QS_TIME'], Y, color='k')
        
        # ssubDF.loc[:,'corrected_angle_diff'] = y[:,0]
        # ssubDF.loc[:,'corrected_angle'] = ssubDF['Wind Direction (QuikSCAT)'] - ssubDF['corrected_angle_diff']

        # Ux_TAO = ssubDF['Neutral Wind Speed at 10m (TAO)'] * np.cos(np.deg2rad(ssubDF['corrected_angle']))
        # Uy_TAO = ssubDF['Neutral Wind Speed at 10m (TAO)'] * np.sin(np.deg2rad(ssubDF['corrected_angle']))
        # subDF.loc[subDF['Deployment index'] == dep, 'Zonal Neutral Wind Speed (TAO) aft. correc.'] = Ux_TAO
        # subDF.loc[subDF['Deployment index'] == dep, 'Meridional Neutral Wind Speed (TAO) aft. correc.'] = Uy_TAO
        subDF.loc[subDF['Deployment index'] == dep, 'corrected_angle_diff'] = y[:,0]
        
    ax = axes[0]
    ax.scatter(subDF['QS_TIME'], subDF['ratio'], c = 'r', s = 0.2)
    ax.scatter(subDF2['QS_TIME'], subDF2['ratio'], c = 'grey', s = 0.4)
    
    ax = axes[1]
    ax.scatter(subDF['QS_TIME'], subDF['angle_diff'], c = 'r', s = 0.2)
    ax.scatter(subDF2['QS_TIME'], subDF2['angle_diff'], c = 'grey', s = 0.4)

    ax = axes[2]
    ax.scatter(subDF['QS_TIME'], subDF['corrected_angle_diff'], c = 'r', s = 0.2)
    #ax.scatter(subDF2['QS_TIME'], subDF2['angle_diff'], c = 'grey', s = 0.4)

    
    for i in range(len(startDates)):
        colorR =list(np.random.choice(range(256), size=3)/256)
        
        axes[0].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
        axes[1].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
        axes[2].axvspan(startDates[i], endDates[i], 
                   alpha=0.05, color=colorR)
    
    axes[0].set_ylim(0,2)
    axes[1].set_ylim(-50,50)
    axes[2].set_ylim(-50,50)
    
    axes[0].set_title('Ratio of wind speeds (QS/TAO)')
    axes[1].set_title('Difference in wind direction (QS - TAO)')
    axes[2].set_title('Difference in wind direction (QS - TAO) after correction from robust regression fit')

    axes[0].grid(axis='y')
    axes[1].grid(axis='y')
    axes[2].grid(axis='y')
    
    if lat < 0:
        latUnits = 'S'
        lat = abs(lat)
    else:
        latUnits = 'N'
    
    if lon < 0:
        lonUnits = 'W'
        lon = abs(lon) # 360
    else:
        lonUnits = 'E'
    title = f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_{suffix}.png'
    plt.savefig(title, dpi = 100)
    plt.close()

In [10]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)

In [11]:
subDF = df.loc[df['predicted_rain_flag'] == 0]
subDF = subDF.loc[subDF['label'] == 0]
# for i in range(ntasks):
#     lat = taskList[i][0]
#     lon = taskList[i][1]
#     plotSusanLikeWithoutCorrectedAngle(subDF, lat, lon, suffix='dryAndGoodWithoutCorrection')

for i in range(ntasks):
    lat = taskList[i][0]
    lon = taskList[i][1]
    plotSusanLikeWithCorrectedAngle(subDF, lat, lon, suffix='dryAndGoodWithAngleCorrection')

for i in range(ntasks):
    lat = taskList[i][0]
    lon = taskList[i][1]
    plotSusanLikeWithCorrectedAngle(subDF, lat, lon, suffix='dryAndGoodWithAngleCorrection')

/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/4274004778.py:17: Runtime

In [12]:
# subDF = df.loc[df['predicted_rain_flag'] == 0]
# subDF = subDF.loc[subDF['label'] == 0]
# for i in range(ntasks):
#     lat = taskList[i][0]
#     lon = taskList[i][1]
#     plotSusanLikeWithoutCorrectedAngle(subDF, lat, lon, suffix='dryAndGoodWithoutCorrection')

# for i in range(ntasks):
#     lat = taskList[i][0]
#     lon = taskList[i][1]
#     plotSusanLikeWithCorrectedAngle(subDF, lat, lon, suffix='dryAndGoodWithAngleCorrection')

for i in range(ntasks):
    lat = taskList[i][0]
    lon = taskList[i][1]
    plotSusanLikeWithCorrectedAngleGreyBad(df, lat, lon, suffix='goodRedBadGrey')

/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: divide by zero encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: invalid value encountered in divide
  r = QS/TAO
/tmp/ipykernel_856142/7006814.py:17: RuntimeWarning: divide by zero encounter